## More data Processing with Pandas
- Reading assignments
    - Chapter eight: Data Wrangling: Join, Combine, Reshape
    - Chapter ten: Data Aggregation and group operations
    - Chapter twelve: Advanced pandas
- Merging DataFrames
- Pandas Idiom
- Group by
- Scales
- Pivot Table
- Date/Time functionality

In [1]:
# Aqui estaremos usando las teorias de conjuntos que se conocen de los Diagramas de Venn.
# Este background se traducira utilizando Pandas. Hay que reconocer las distitnas convenciones 
# como outer-full-join, inner-join, etc y su relacion con las uniones e intercepciones

import pandas as pd
# Primero creamos dos DataFrames, staff y students
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR',
                          'Name': 'Sally', 'Role': 'Course liasion',
                          'Name': 'James', 'Role': 'Grader'}])

In [2]:
# Nos quedamos en 3:24